<a href="https://colab.research.google.com/github/nitron-alpha-kplr/Cloner-Netflix-avec-DataStax-DB-et-GraphQL/blob/main/Copie_de_4_Scrape_Car_Attributs_Using_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Récupérer les attributs d'une voiture à l'aide de Requests**





- Nous allons extraire des données des sites web d'un concessionnaire automobile : nous allons extraire - la marque - le modèle - le type de carburant - les kilomètres - l'année du modèle - le prix.

![image](https://user-images.githubusercontent.com/123748165/224308731-cb43fb4b-69f9-4d5d-b52f-1d2b157aaae5.png)

#### 1 - Imports

In [ ]:
import requests 
import pandas as pd 
import sqlalchemy

#### 2 - CURL 

**Comment afficher les en-têtes HTTP ? [Y compris les cookies]**

- Avant de pouvoir utiliser les en-têtes dans notre code, nous devons être en mesure de les récupérer quelque part. Pour ce faire, utilisons notre propre navigateur et allons sur le site web cible. 
- Pour notre exemple, allons sur https://www.cazoo.co.uk/cars/?page=2 > clic droit > inspecter pour ouvrir les outils de développement.

- nous allons naviguer dans l'onglet Réseau et rechercher la requête "?page 2".

![image](https://user-images.githubusercontent.com/123748165/224308744-61c1ed97-1680-47bf-a531-0f0daa24f5ce.png)

- Clicking on the file we’ll open by default the Headers tab and by scrolling down we’ll see the Request Headers section.

![image](https://user-images.githubusercontent.com/123748165/224308758-b28b3301-4f75-42ac-a682-953790c4405c.png)


***Headers Arguments***

- **User-Agent** : Il s'agit probablement de l'en-tête le plus important, car il identifie "le type d'application, le système d'exploitation, le fournisseur du logiciel ou la version du logiciel de l'agent utilisateur demandeur".
le fournisseur du logiciel ou la version logicielle de l'agent utilisateur du logiciel demandeur", ce qui en fait le premier contrôle effectué par la plupart des serveurs.

- **Accept-Language** : Bien qu'elle ne soit pas toujours nécessaire, cette fonction indique au serveur la version linguistique des données à fournir. Lorsqu'il y a une grande différence entre chaque demande du point de vue de la langue, cela peut indiquer au serveur qu'un robot est impliqué.

- **Referer** : L'en-tête HTTP Referer indique au serveur la page d'où provient l'utilisateur. Bien qu'il soit principalement utilisé pour le suivi, il peut également nous aider à imiter le comportement d'un utilisateur organique, par exemple en indiquant au serveur que nous venons d'un moteur de recherche comme Google.

- **Cookie** : Les cookies permettent aux serveurs de communiquer à l'aide d'un petit morceau de données, mais que se passe-t-il lorsque le serveur envoie un cookie mais que le navigateur ne le stocke pas et ne le renvoie pas lors de la requête suivante ? Les cookies peuvent également être utilisés pour identifier si la demande provient d'un utilisateur réel ou d'un robot.
- **sec-fetch-site** : cet en-tête indique au serveur si une demande de ressource provient de la même origine, du même site, d'un site différent ou s'il s'agit d'une demande "initiée par l'utilisateur". Le serveur peut alors utiliser cette information pour décider si la demande doit être autorisée.
- **sec-fetch-mode** : permet à un serveur de faire la distinction entre : les demandes provenant d'un utilisateur naviguant entre des pages HTML, et les demandes de chargement d'images et d'autres ressources. Par exemple, cet en-tête contiendrait navigate pour les demandes de navigation de haut niveau, tandis que no-cors serait utilisé pour le chargement d'une image.

- **sec-fetch-Dest** : permet aux serveurs de déterminer s'ils doivent répondre à une demande en fonction de son utilisation prévue.

In [ ]:
headers = {
    'authority': 'www.cazoo.co.uk',
    'sec-ch-ua': '^\\^',
    'cazoo-language': 'en',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'cazoo-market': 'gb',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.cazoo.co.uk/cars/?page=2',
    'accept-language': 'en-US,en;q=0.9,de-DE;q=0.8,de;q=0.7',
    'cookie': 'base_device_id=d0a0a3f4-6853-469a-920f-6050e04a23d0; base_session_count=1; base_session_id=aa7da8a1-20b0-44f4-b994-7b19b379af45',
}

params = (
    ('sort', 'price-asc'),
)

response = requests.get('https://www.cazoo.co.uk/api/search', headers=headers, params=params)


3 - Check Status Code

In [ ]:
response

<Response [200]>

#### 4 - Créer un objet Json

In [ ]:
results_json = response.json()
results_json

{'results': [{'createdAt': '2022-11-08T08:38:09.112Z',
   'id': 'd70f47aa-7575-550c-99f5-9be578268782',
   'make': 'Kia',
   'makeIdentifier': 'kia',
   'model': 'Picanto',
   'modelIdentifier': 'picanto',
   'modelYear': 2015,
   'displayVariant': '1.0 SR7',
   'mileage': 36057,
   'registrationYear': 2016,
   'vrm': 'DF16YTA',
   'isForSubscription': False,
   'isForPurchase': True,
   'fuelType': {'description': 'Petrol',
    'isEcoFriendly': False,
    'isPlugin': False},
   'images': {'main': {'url': 'https://media.cazoo.com/image/upload/c_scale,f_auto,h_405,q_auto,w_720/cazoo-imagery/car_gallery_images/KNABX311LGT188402/402fe397-dd90-4d98-b5e8-516130dfcda1.jpg'}},
   'isPromoted': True,
   'odometerReading': {'unit': 'mile', 'value': 36057},
   'tradingMarket': 'gb',
   'vehicleType': 'Car',
   'gearbox': 'Manual',
   'gallery': {'prefix': 'https://media.cazoo.com/image/upload/c_scale,f_auto,h_405,q_auto,w_720/cazoo-imagery/',
    'items': ['car_gallery_images/KNABX311LGT188402/4

#### 5 - Output Keys

In [ ]:
results_json.keys()

dict_keys(['results', 'filters', 'pagination'])

In [ ]:
listing = results_json['results']

#### 6 - Trouvez vos données 
- marque
- modèle
- type de carburant
- Kilométrage
- année du modèle
- prix

In [ ]:
# brand
listing[0]['make']

'Kia'

In [ ]:
# model
listing[0]['model']

'Picanto'

In [ ]:
# fuel type
listing[0]['fuelType']['description']

'Petrol'

In [ ]:
#miles
listing[0]['mileage']

36057

In [ ]:
# model year
listing[0]['modelYear']

2015

In [ ]:
# price
listing[0]['pricing']['fullPrice']['value']

6500

#### 7 - Mettre tout ensemble - Boucler les résultats et ajouter des données dans une liste

In [ ]:
# empty lists
brand = []
model = []
fuel_type = []
miles = []
model_year = []
price = []

for result in listing:
    
    # brand
    brand.append(result['make'])
    
    # model 
    model.append(result['model'])
    
    # fuel type
    fuel_type.append(result['fuelType']['description'])
    
    # miles
    miles.append(result['mileage'])
    
    #model year
    model_year.append(result['modelYear'])
    
    # price
    price.append(result['pricing']['fullPrice']['value'])

#### 8 - Pandas Dataframe - Single Page

In [ ]:
car_df = pd.DataFrame({'Brand': brand, 'Model':model, 'Fuel Type': fuel_type, 'Mileage':miles,
                      'Model Year': model_year, 'Price':price})

In [ ]:
car_df

,Brand,Model,Fuel Type,Mileage,Model Year,Price
0,Kia,Picanto,Petrol,36057,2015,6500
1,Peugeot,108,Petrol,35847,2016,6800
2,Peugeot,108,Petrol,62681,2016,6850
3,Fiat,500,Petrol,41216,2015,6950
4,Vauxhall,Viva,Petrol,44244,2016,7150
5,Vauxhall,Corsa,Petrol,30278,2017,7150
6,Vauxhall,ADAM,Petrol,36676,2017,7200
7,Fiat,500,Petrol,35534,2015,7250
8,Citroen,C1,Petrol,45950,2015,7350
9,Vauxhall,Viva,Petrol,9664,2017,7350


In [ ]:
car_df.to_csv('data.csv')

#### 9 - Scraping Plusieurs Pages (ici : 20 pages)

In [ ]:
headers = {
    'authority': 'www.cazoo.co.uk',
    'sec-ch-ua': '^\\^',
    'cazoo-language': 'en',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'cazoo-market': 'gb',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.cazoo.co.uk/cars/',
    'accept-language': 'en-US,en;q=0.9,de-DE;q=0.8,de;q=0.7',
    'cookie': 'base_device_id=d0a0a3f4-6853-469a-920f-6050e04a23d0; base_session_count=1',
}

# empty lists
brand = []
model = []
fuel_type = []
miles = []
model_year = []
price = []

for i in range(1,21):

    params = (
        ('page', str(i)),
        ('sort', 'price-asc'),
    )

    response = requests.get('https://www.cazoo.co.uk/api/search', headers=headers, params=params)

    results_json = response.json()
    
    listing = results_json['results']
    
    for result in listing:
    
        # brand
        try:
            brand.append(result['make'])
        except:
            brand.append('')

        # model 
        try:
            model.append(result['model'])
        except:
            model.append('')

        # fuel type
        try:
            fuel_type.append(result['fuelType']['description'])
        except:
            fuel_type.append('')

        # miles
        try:
            miles.append(result['mileage'])
        except:
            miles.append('')

        #model year
        try:
            model_year.append(result['modelYear'])
        except:
            model_year

        # price
        try:
            price.append(result['pricing']['fullPrice']['value'])
        except:
            price.append('')
        

car_df_multiple = pd.DataFrame({'Brand': brand, 'Model':model, 'Fuel Type': fuel_type, 'Mileage':miles,
                      'Model Year': model_year, 'Price':price})   

car_df_multiple


,Brand,Model,Fuel Type,Mileage,Model Year,Price
0,Kia,Picanto,Petrol,36057,2015,6500
1,Peugeot,108,Petrol,35847,2016,6800
2,Peugeot,108,Petrol,62681,2016,6850
3,Fiat,500,Petrol,41216,2015,6950
4,Vauxhall,Viva,Petrol,44244,2016,7150
...,...,...,...,...,...,...
955,Audi,A3,Diesel,96053,2018,13100
956,Ford,Fiesta,Petrol,33566,2020,13100
957,Volkswagen,Golf,Diesel,70607,2020,13100
958,Volkswagen,Polo,Petrol,19024,2019,13100


In [ ]:
car_df_multiple.to_csv('data_multiple.csv')